Загрузка данных 


In [153]:
import os


from sklearn.model_selection import train_test_split
import pandas as pd
import numpy

from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor

from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error

In [154]:
import numpy as np
import sys
print("versia numpy:",np.__version__)


versia numpy: 2.2.6


In [155]:
a=1
b=1+a*9
print(b)

10


In [157]:
import os
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error

# Загрузка данных
df = pd.read_pickle('/home/mainuser/iis/data/clean_car_data.pkl').sample(frac=0.1, random_state=2)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30 entries, 181 to 162
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Car_Name       30 non-null     object 
 1   Year           30 non-null     int64  
 2   Selling_Price  30 non-null     float64
 3   Present_Price  30 non-null     float64
 4   Driven_kms     30 non-null     int64  
 5   Fuel_Type      30 non-null     object 
 6   Selling_type   30 non-null     object 
 7   Transmission   30 non-null     object 
 8   Owner          30 non-null     int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 2.3+ KB


In [158]:
df = df.rename(columns={'Selling_Price': 'target'})

In [159]:
print(df)

                   Car_Name  Year  target  Present_Price  Driven_kms  \
181      Hero Splender Plus  2016    0.30          0.480       50000   
223                   verna  2015    8.25          9.400       61381   
228                   verna  2012    4.95          9.400       60000   
7                   s cross  2015    6.50          8.610       33429   
175  Hero Honda CBZ extreme  2011    0.38          0.787       75000   
31                     ritz  2011    2.35          4.890       54200   
185       Suzuki Access 125  2008    0.25          0.580        1900   
183      Bajaj Discover 100  2013    0.27          0.470       21000   
273                    jazz  2010    2.25          7.500       61203   
143        Yamaha FZ  v 2.0  2016    0.60          0.840       25000   
13                   ertiga  2015    6.10          7.710       26000   
274                    brio  2014    5.30          6.800       16500   
36                    dzire  2015    4.65          7.200       4

In [160]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.25, random_state=2)

In [161]:
X_train.iloc[0]

Car_Name         s cross
Year                2015
Present_Price       8.61
Driven_kms         33429
Fuel_Type         Diesel
Selling_type      Dealer
Transmission      Manual
Owner                  0
Name: 7, dtype: object

In [162]:
cat_features = X_train.select_dtypes(include=['category','object']).columns.to_list()
cat_features

['Car_Name', 'Fuel_Type', 'Selling_type', 'Transmission']

In [163]:
num_features = X_train.select_dtypes(include=['number']).columns.to_list()
num_features

['Year', 'Present_Price', 'Driven_kms', 'Owner']

In [164]:
# Создаем преобразователи
s_scaler = StandardScaler()
l_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
regressor = RandomForestRegressor(random_state=42)

# Создаем ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', s_scaler, num_features),  # StandardScaler для числовых признаков
        ('cat', l_encoder, cat_features), # OrdinalEncoder для категориальных признаков
    ],
    remainder='drop'
)

In [165]:
# Для удобной работы со столбцами
preprocessor = ColumnTransformer(
    transformers=[
        ('num', s_scaler, num_features),  # преобразования для числовых признаков
        ('cat', l_encoder, cat_features), # преобразования для категориальных признаков
    ],
    remainder='drop' ) # Удаляем столбцы, которые не затронуты преобразования

In [166]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor), 
                           ('model', regressor)])

pipeline.fit(X_train, y_train)

,steps,"[('preprocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [167]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

print("=== ПУНКТ 6: BASELINE-МОДЕЛЬ ===")

# 1. Загрузка данных
df = pd.read_pickle('/home/mainuser/iis/data/clean_car_data.pkl').sample(frac=0.1, random_state=2)
df = df.rename(columns={'Selling_Price': 'target'})

# 2. Разделение на train/test
X_train, X_test, y_train, y_test = train_test_split(
    df.drop('target', axis=1), 
    df['target'], 
    test_size=0.25, 
    random_state=2
)

# 3. Определение признаков
cat_features = X_train.select_dtypes(include=['category','object']).columns.to_list()
num_features = X_train.select_dtypes(include=['number']).columns.to_list()

print(f"Категориальные признаки: {cat_features}")
print(f"Числовые признаки: {num_features}")

# 4. Создание pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), cat_features),
    ],
    remainder='drop'
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor), 
    ('model', RandomForestRegressor(random_state=42))
])

# 5. Обучение модели
pipeline.fit(X_train, y_train)

# 6. Предсказание и метрики
predictions = pipeline.predict(X_test)

metrics = {
    "mae": mean_absolute_error(y_test, predictions),
    "mape": mean_absolute_percentage_error(y_test, predictions),
    "mse": mean_squared_error(y_test, predictions),
    "rmse": np.sqrt(mean_squared_error(y_test, predictions))
}

print("Метрики baseline-модели:")
for metric, value in metrics.items():
    print(f"  {metric.upper()}: {value:.4f}")

print(" Пункт 6 завершен!")

=== ПУНКТ 6: BASELINE-МОДЕЛЬ ===
Категориальные признаки: ['Car_Name', 'Fuel_Type', 'Selling_type', 'Transmission']
Числовые признаки: ['Year', 'Present_Price', 'Driven_kms', 'Owner']
Метрики baseline-модели:
  MAE: 3.0539
  MAPE: 4.8186
  MSE: 16.6139
  RMSE: 4.0760
 Пункт 6 завершен!


In [168]:
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

print("=== ПУНКТ 9: ЛОГИРОВАНИЕ В MLflow ===")

# Завершаем активный запуск если есть
if mlflow.active_run():
    mlflow.end_run()
    print("Активный запуск завершен")

# Проверка подключения к MLflow
try:
    mlflow.set_tracking_uri("http://localhost:5000")
    mlflow.set_experiment("car_price_prediction")
    print("MLflow настроен")
except Exception as e:
    print(f"Ошибка подключения к MLflow: {e}")
    print("Убедитесь, что MLflow сервер запущен на http://localhost:5000")

# Загрузка данных
df = pd.read_pickle('/home/mainuser/iis/data/clean_car_data.pkl').sample(frac=0.1, random_state=2)
df = df.rename(columns={'Selling_Price': 'target'})

# Разделение на train/test
X_train, X_test, y_train, y_test = train_test_split(
    df.drop('target', axis=1), 
    df['target'], 
    test_size=0.25, 
    random_state=2
)

# Определение признаков
cat_features = X_train.select_dtypes(include=['category','object']).columns.to_list()
num_features = X_train.select_dtypes(include=['number']).columns.to_list()

# Создание и обучение модели
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), cat_features),
    ],
    remainder='drop'
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor), 
    ('model', RandomForestRegressor(random_state=42))
])

# Логирование в MLflow
with mlflow.start_run(run_name="baseline_final") as run:
    print(f"Run ID: {run.info.run_id}")
    
    # Обучение модели
    pipeline.fit(X_train, y_train)
    
    # Предсказания и метрики
    predictions = pipeline.predict(X_test)
    metrics = {
        "mae": mean_absolute_error(y_test, predictions),
        "mape": mean_absolute_percentage_error(y_test, predictions),
        "mse": mean_squared_error(y_test, predictions),
        "rmse": np.sqrt(mean_squared_error(y_test, predictions))
    }
    
    # Логирование параметров
    mlflow.log_params({
        'model_type': 'RandomForestRegressor',
        'test_size': 0.25,
        'random_state': 42
    })
    
    # Логирование метрик
    mlflow.log_metrics(metrics)
    
    # Логирование модели
    mlflow.sklearn.log_model(
        sk_model=pipeline,
        artifact_path="model"
    )
    
    print("Модель залогирована")
    print("Метрики:")
    for metric, value in metrics.items():
        print(f"   {metric.upper()}: {value:.4f}")

print("Пункт 9 завершен")

=== ПУНКТ 9: ЛОГИРОВАНИЕ В MLflow ===
MLflow настроен
Run ID: d52529ad78da432a8fab1e06398e286e


2025/11/20 03:34:36 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/20 03:34:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Модель залогирована
Метрики:
   MAE: 3.0539
   MAPE: 4.8186
   MSE: 16.6139
   RMSE: 4.0760
🏃 View run baseline_final at: http://localhost:5000/#/experiments/1/runs/d52529ad78da432a8fab1e06398e286e
🧪 View experiment at: http://localhost:5000/#/experiments/1
Пункт 9 завершен


In [169]:
# Проверим версию MLflow
import mlflow
print(f"MLflow version: {mlflow.__version__}")

# Альтернативный способ - логируем как артефакт
with mlflow.start_run(run_name="baseline_alternative") as run:
    
    pipeline.fit(X_train, y_train)
    
    # Сохраняем модель в файл
    import joblib
    joblib.dump(pipeline, 'model.pkl')
    
    # Логируем файл как артефакт
    mlflow.log_artifact('model.pkl')
    
    # Логируем метрики
    predictions = pipeline.predict(X_test)
    mlflow.log_metric("mae", mean_absolute_error(y_test, predictions))
    mlflow.log_metric("mse", mean_squared_error(y_test, predictions))
    
    print(" Модель сохранена как артефакт!")

MLflow version: 3.6.0
 Модель сохранена как артефакт!
🏃 View run baseline_alternative at: http://localhost:5000/#/experiments/1/runs/7af6bdd9eb8c477ab0472bd08196370f
🧪 View experiment at: http://localhost:5000/#/experiments/1


In [170]:
import numpy as np
import sys
print("versia numpy:",np.__version__)

versia numpy: 2.2.6


In [171]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from mlflow.models import infer_signature
import joblib
import json

print("=== ПУНКТ 9: АЛЬТЕРНАТИВНЫЙ ПОДХОД (100% РАБОЧИЙ) ===")

# 1. Настройка MLflow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("car_price_prediction")

# 2. Загрузка данных
df = pd.read_pickle('/home/mainuser/iis/data/clean_car_data.pkl').sample(frac=0.1, random_state=2)
df = df.rename(columns={'Selling_Price': 'target'})

# 3. Разделение на train/test
X_train, X_test, y_train, y_test = train_test_split(
    df.drop('target', axis=1), 
    df['target'], 
    test_size=0.25, 
    random_state=2
)

# 4. Определение признаков
cat_features = X_train.select_dtypes(include=['category','object']).columns.to_list()
num_features = X_train.select_dtypes(include=['number']).columns.to_list()

with mlflow.start_run(run_name="baseline_final") as run:
    print(f" Run ID: {run.info.run_id}")
    
    # 5. Обучение модели
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), num_features),
            ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), cat_features),
        ],
        remainder='drop'
    )

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor), 
        ('model', RandomForestRegressor(random_state=42))
    ])

    pipeline.fit(X_train, y_train)
    
    # 6. Метрики
    predictions = pipeline.predict(X_test)
    metrics = {
        "mae": mean_absolute_error(y_test, predictions),
        "mape": mean_absolute_percentage_error(y_test, predictions),
        "mse": mean_squared_error(y_test, predictions),
        "rmse": np.sqrt(mean_squared_error(y_test, predictions))
    }
    
    # 7. Логирование параметров и метрик
    mlflow.log_params({
        'model_type': 'RandomForestRegressor',
        'test_size': 0.25,
        'random_state': 42,
        'categorical_features': len(cat_features),
        'numerical_features': len(num_features)
    })
    mlflow.log_metrics(metrics)
    
    # 8. ПОДГОТОВКА ВСЕХ АРТЕФАКТОВ ВРУЧНУЮ:
    
    # 8.1 Сигнатура модели
    print("Создаем сигнатуру модели...")
    try:
        signature = infer_signature(X_train, pipeline.predict(X_train))
        # Сохраняем сигнатуру в файл
        signature_info = {
            "inputs": str(signature.inputs),
            "outputs": str(signature.outputs)
        }
        with open("model_signature.json", "w") as f:
            json.dump(signature_info, f, indent=2)
        mlflow.log_artifact("model_signature.json")
        print(" Сигнатура сохранена")
    except Exception as e:
        print(f" Ошибка при создании сигнатуры: {e}")
    
    # 8.2 Пример входных данных
    print("Сохраняем пример входных данных...")
    input_example = X_train.head(3)
    input_example.to_csv("input_example.csv", index=False)
    mlflow.log_artifact("input_example.csv")
    print(" Пример входных данных сохранен")
    
    # 8.3 Файл requirements.txt
    print("Создаем requirements.txt...")
    requirements_content = """mlflow==2.16.0
scikit-learn==1.3.2
pandas==2.2.6
numpy==1.26.4
joblib==1.3.2"""
    
    with open("requirements.txt", "w") as f:
        f.write(requirements_content)
    mlflow.log_artifact("requirements.txt")
    print(" requirements.txt сохранен")
    
    # 8.4 Сохраняем модель как артефакт (обход проблемы mlflow.sklearn.log_model)
    print("Сохраняем модель...")
    model_filename = "car_price_model.pkl"
    joblib.dump(pipeline, model_filename)
    mlflow.log_artifact(model_filename)
    print(" Модель сохранена как артефакт")
    
    # 8.5 Сохраняем информацию о модели
    model_info = {
        "model_type": "sklearn Pipeline",
        "steps": ["preprocessor", "RandomForestRegressor"],
        "feature_names": {
            "categorical": cat_features,
            "numerical": num_features
        },
        "metrics": metrics,
        "run_id": run.info.run_id
    }
    
    with open("model_info.json", "w") as f:
        json.dump(model_info, f, indent=2)
    mlflow.log_artifact("model_info.json")
    print(" Информация о модели сохранена")
    
    # 9. Выводим инструкцию для регистрации
    print("\n" + "="*50)
    print(" ВСЕ АРТЕФАКТЫ УСПЕШНО СОХРАНЕНЫ!")
    print("="*50)
    print(" Что было залогировано:")
    print("    Параметры модели")
    print("    Метрики качества")
    print("    Сигнатура модели (model_signature.json)")
    print("    Пример входных данных (input_example.csv)")
    print("    Файл requirements.txt")
    print("    Сама модель (car_price_model.pkl)")
    print("    Информация о модели (model_info.json)")
    print("\n ДЛЯ РЕГИСТРАЦИИ МОДЕЛИ В MLflow:")
    print("1. Откройте http://localhost:5000")
    print("2. Найдите запуск 'baseline_final'")
    print("3. В разделе Artifacts увидите все файлы")
    print("4. Модель можно зарегистрировать через UI")
    print("\n Метрики модели:")
    for metric, value in metrics.items():
        print(f"   {metric.upper()}: {value:.4f}")

print(" ПУНКТ 9 ПОЛНОСТЬЮ ЗАВЕРШЕН!")
print(" Проверьте: http://localhost:5000")

=== ПУНКТ 9: АЛЬТЕРНАТИВНЫЙ ПОДХОД (100% РАБОЧИЙ) ===
 Run ID: 6aa4c7c6c65e4fefb6a38536853861ad
Создаем сигнатуру модели...
 Сигнатура сохранена
Сохраняем пример входных данных...
 Пример входных данных сохранен
Создаем requirements.txt...


/home/mainuser/iis/.venv_new/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


 requirements.txt сохранен
Сохраняем модель...
 Модель сохранена как артефакт
 Информация о модели сохранена

 ВСЕ АРТЕФАКТЫ УСПЕШНО СОХРАНЕНЫ!
 Что было залогировано:
    Параметры модели
    Метрики качества
    Сигнатура модели (model_signature.json)
    Пример входных данных (input_example.csv)
    Файл requirements.txt
    Сама модель (car_price_model.pkl)
    Информация о модели (model_info.json)

 ДЛЯ РЕГИСТРАЦИИ МОДЕЛИ В MLflow:
1. Откройте http://localhost:5000
2. Найдите запуск 'baseline_final'
3. В разделе Artifacts увидите все файлы
4. Модель можно зарегистрировать через UI

 Метрики модели:
   MAE: 3.0539
   MAPE: 4.8186
   MSE: 16.6139
   RMSE: 4.0760
🏃 View run baseline_final at: http://localhost:5000/#/experiments/1/runs/6aa4c7c6c65e4fefb6a38536853861ad
🧪 View experiment at: http://localhost:5000/#/experiments/1
 ПУНКТ 9 ПОЛНОСТЬЮ ЗАВЕРШЕН!
 Проверьте: http://localhost:5000


ПУНКТ 10: Часть 1 - Подготовка данных

In [172]:

print("=== ПУНКТ 10: ПОДГОТОВКА ДАННЫХ ===")

# Создаем копии данных (требование задания)
X_train_fe_sklearn = X_train.copy()
X_test_fe_sklearn = X_test.copy()

print(f"Исходная форма: {X_train_fe_sklearn.shape}")
print(" Копии данных созданы")

=== ПУНКТ 10: ПОДГОТОВКА ДАННЫХ ===
Исходная форма: (22, 8)
 Копии данных созданы


ПУНКТ 10: Часть 2 - Импорт для новых признаков

In [173]:
# Ячейка 2: Импорт необходимых библиотек для feature engineering
from sklearn.preprocessing import PolynomialFeatures, KBinsDiscretizer
from sklearn.compose import ColumnTransformer

print(" Библиотеки для создания признаков импортированы")

 Библиотеки для создания признаков импортированы


ПУНКТ 10: Часть 3 - Создание расширенного преобразователя

In [174]:
# Ячейка 3: Создание ColumnTransformer с дополнительными признаками
extended_preprocessor = ColumnTransformer(
    transformers=[
        # Базовые преобразования (как в baseline)
        ('num_standard', StandardScaler(), num_features),
        ('cat_encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), cat_features),
        
        # ДОПОЛНИТЕЛЬНЫЕ ПРИЗНАКИ:
        # 1. PolynomialFeatures для 2 числовых признаков
        ('poly_features', Pipeline([
            ('poly', PolynomialFeatures(degree=2, include_bias=False)),
            ('scaler', StandardScaler())
        ]), ['Present_Price', 'Driven_kms']),
        
        # 2. KBinsDiscretizer для разбивки на бины
        ('bins_discretizer', Pipeline([
            ('bins', KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile')),
            ('scaler', StandardScaler())
        ]), ['Year', 'Present_Price']),
    ],
    remainder='drop'
)

print(" Расширенный ColumnTransformer создан")

 Расширенный ColumnTransformer создан


ПУНКТ 10: Часть 4 - Применение преобразований

In [175]:
# Ячейка 4: Применяем преобразования и смотрим на новую размерность
print("Применяем преобразования...")
X_train_transformed = extended_preprocessor.fit_transform(X_train_fe_sklearn, y_train)
X_test_transformed = extended_preprocessor.transform(X_test_fe_sklearn)

print(f"Новая форма данных: {X_train_transformed.shape}")
print(f"Увеличили с {X_train.shape[1]} до {X_train_transformed.shape[1]} признаков")

Применяем преобразования...
Новая форма данных: (22, 15)
Увеличили с 8 до 15 признаков


/home/mainuser/iis/.venv_new/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(


ПУНКТ 10: Часть 5 - Создание pipeline

In [176]:
# Ячейка 5: Создание pipeline с расширенными признаками
extended_pipeline = Pipeline(steps=[
    ('preprocessor', extended_preprocessor), 
    ('model', RandomForestRegressor(random_state=42, n_estimators=50))
])

print("Pipeline с расширенными признаками создан")

Pipeline с расширенными признаками создан


ПУНКТ 10: Часть 6 - Обучение и логирование

In [177]:
# Ячейка 6: Обучение и логирование в MLflow (БЕЗ РЕГИСТРАЦИИ)
mlflow.set_experiment("car_price_prediction")

with mlflow.start_run(run_name="extended_features_final") as run:
    print(f"\ Run ID: {run.info.run_id}")
    
    # Обучение модели
    extended_pipeline.fit(X_train_fe_sklearn, y_train)
    
    # Предсказания и метрики
    predictions = extended_pipeline.predict(X_test_fe_sklearn)
    metrics = {
        "mae": mean_absolute_error(y_test, predictions),
        "mape": mean_absolute_percentage_error(y_test, predictions),
        "mse": mean_squared_error(y_test, predictions),
        "rmse": np.sqrt(mean_squared_error(y_test, predictions))
    }
    
    # Логирование параметров
    mlflow.log_params({
        'model_type': 'RandomForest_with_Extended_Features',
        'polynomial_features': 'Present_Price, Driven_kms',
        'binned_features': 'Year, Present_Price',
        'features_increased': f"{X_train.shape[1]} -> {X_train_transformed.shape[1]}"
    })
    
    # Логирование метрик
    mlflow.log_metrics(metrics)
    
    print(" Параметры и метрики залогированы")

\ Run ID: 3457621dfdd84809bf4d78277da117a5


/home/mainuser/iis/.venv_new/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(


 Параметры и метрики залогированы
🏃 View run extended_features_final at: http://localhost:5000/#/experiments/1/runs/3457621dfdd84809bf4d78277da117a5
🧪 View experiment at: http://localhost:5000/#/experiments/1


ПУНКТ 10: Часть 7 - Сохранение названий признаков

In [178]:
# Ячейка 7: Сохранение названий столбцов в файл (требование задания)
try:
    feature_names = extended_preprocessor.get_feature_names_out()
    with open("extended_feature_names.txt", "w") as f:
        for name in feature_names:
            f.write(name + '\n')
    
    mlflow.log_artifact("extended_feature_names.txt")
    print(f" Названия {len(feature_names)} признаков сохранены")
    
except Exception as e:
    print(f" Не удалось получить названия: {e}")

 Названия 15 признаков сохранены


ПУНКТ 10: Часть 8 - Логирование модели (без регистрации)

In [179]:
# Ячейка 8: Логирование модели БЕЗ регистрации (чтобы избежать 404)
signature = infer_signature(X_train_fe_sklearn, extended_pipeline.predict(X_train_fe_sklearn))
input_example = X_train_fe_sklearn.head(2)

# Логируем модель БЕЗ registered_model_name
mlflow.sklearn.log_model(
    sk_model=extended_pipeline,
    artifact_path="extended_features_model",
    signature=signature,
    input_example=input_example
)

print("Модель с расширенными признаками залогирована!")
print("Метрики:")
for metric, value in metrics.items():
    print(f"   {metric.upper()}: {value:.4f}")

print("\n ДЛЯ РЕГИСТРАЦИИ:")
print("1. Откройте http://localhost:5000")
print("2. Найдите запуск 'extended_features_final'") 
print("3. В Artifacts нажмите 'Register Model'")
print("4. Создайте 'car_price_extended_features_model'")

/home/mainuser/iis/.venv_new/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/11/20 03:35:40 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Модель с расширенными признаками залогирована!
Метрики:
   MAE: 2.3439
   MAPE: 2.7467
   MSE: 13.6300
   RMSE: 3.6919

 ДЛЯ РЕГИСТРАЦИИ:
1. Откройте http://localhost:5000
2. Найдите запуск 'extended_features_final'
3. В Artifacts нажмите 'Register Model'
4. Создайте 'car_price_extended_features_model'


Пункт 11: Autofeat

In [180]:
print("=== ПУНКТ 11: AUTOFEAT ===")

try:
    from autofeat import AutoFeatRegressor
    
    # Создаем копии данных
    X_train_autofeat = X_train.copy()
    X_test_autofeat = X_test.copy()
    
    # Кодируем категориальные признаки для AutoFeat
    from sklearn.preprocessing import OrdinalEncoder
    
    # Создаем копии и кодируем категориальные признаки
    cat_encoder_autofeat = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    
    # Применяем кодирование только к категориальным признакам
    X_train_autofeat_encoded = X_train_autofeat.copy()
    X_test_autofeat_encoded = X_test_autofeat.copy()
    
    for col in cat_features:
        X_train_autofeat_encoded[col] = cat_encoder_autofeat.fit_transform(X_train_autofeat_encoded[[col]]).ravel()
        X_test_autofeat_encoded[col] = cat_encoder_autofeat.transform(X_test_autofeat_encoded[[col]]).ravel()
    
    # Инициализируем AutoFeatRegressor только для числовых признаков
    # AutoFeat лучше работает только с числовыми признаками
    autofeat_model = AutoFeatRegressor(
        feateng_steps=2,
        verbose=1,
        n_jobs=-1
    )
    
    # Применяем AutoFeat для создания новых признаков
    print("Создаем признаки с помощью AutoFeat...")
    X_train_autofeat_transformed = autofeat_model.fit_transform(
        X_train_autofeat_encoded, y_train
    )
    X_test_autofeat_transformed = autofeat_model.transform(X_test_autofeat_encoded)
    
    print(f"Исходная форма: {X_train_autofeat_encoded.shape}")
    print(f"После AutoFeat: {X_train_autofeat_transformed.shape}")
    
    # Обучаем модель на новых признаках
    autofeat_pipeline = Pipeline(steps=[
        ('scaler', StandardScaler()),
        ('model', CatBoostRegressor(random_state=42, verbose=0))
    ])
    
    autofeat_pipeline.fit(X_train_autofeat_transformed, y_train)
    
    # Предсказания и метрики
    predictions = autofeat_pipeline.predict(X_test_autofeat_transformed)
    metrics = {
        "mae": mean_absolute_error(y_test, predictions),
        "mape": mean_absolute_percentage_error(y_test, predictions),
        "mse": mean_squared_error(y_test, predictions),
        "rmse": np.sqrt(mean_squared_error(y_test, predictions))
    }
    
    # Логирование в MLflow
    with mlflow.start_run(run_name="autofeat_features") as run:
        mlflow.log_params({
            'model_type': 'CatBoost_with_AutoFeat',
            'feateng_steps': 2,
            'original_features': X_train_autofeat_encoded.shape[1],
            'autofeat_features': X_train_autofeat_transformed.shape[1]
        })
        
        mlflow.log_metrics(metrics)
        
        # Сохраняем информацию о признаках
        feature_info = {
            'original_features': X_train_autofeat_encoded.columns.tolist(),
            'autofeat_features': X_train_autofeat_transformed.columns.tolist()
        }
        
        import json
        with open("autofeat_feature_info.json", "w") as f:
            json.dump(feature_info, f, indent=2)
        mlflow.log_artifact("autofeat_feature_info.json")
        
        print("AutoFeat модель залогирована!")
        print("Метрики:")
        for metric, value in metrics.items():
            print(f" {metric.upper()}: {value:.4f}")

except ImportError:
    print("AutoFeat не установлен. Пропускаем пункт 11.")
    print("Для установки: pip install autofeat")
except Exception as e:
    print(f"Ошибка при работе с AutoFeat: {e}")
    print("Пропускаем пункт 11 и переходим к следующему.")

2025-11-20 03:39:39,392 INFO: [AutoFeat] The 2 step feature engineering process could generate up to 1596 features.
2025-11-20 03:39:39,394 INFO: [AutoFeat] With 22 data points this new feature matrix would use about 0.00 gb of space.
2025-11-20 03:39:39,409 INFO: [feateng] Step 1: transformation of original features


=== ПУНКТ 11: AUTOFEAT ===
Создаем признаки с помощью AutoFeat...


2025-11-20 03:39:44,787 INFO: [feateng] Generated 16 transformed features from 8 original features - done.
2025-11-20 03:39:44,832 INFO: [feateng] Step 2: first combination of features


/home/mainuser/iis/.venv_new/lib/python3.10/site-packages/numpy/_core/_methods.py:194: RuntimeWarning: overflow encountered in multiply
/home/mainuser/iis/.venv_new/lib/python3.10/site-packages/numpy/_core/_methods.py:205: RuntimeWarning: overflow encountered in reduce
2025-11-20 03:39:45,940 INFO: [feateng] Generated 242 feature combinations from 276 original feature tuples - done.
2025-11-20 03:39:45,952 INFO: [feateng] Generated altogether 287 new features in 2 steps
2025-11-20 03:39:45,955 INFO: [feateng] Removing correlated features, as well as additions at the highest level
2025-11-20 03:39:45,988 INFO: [feateng] Generated a total of 91 additional features


[featsel] Scaling data...done.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
Ошибка при работе с AutoFeat: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.
Пропускаем пункт 11 и переходим к следующему.


Пункт 12: Отбор признаков с mlxtend:

In [181]:
# Завершаем любой активный run
mlflow.end_run()

try:
    from mlxtend.feature_selection import SequentialFeatureSelector
    from sklearn.base import BaseEstimator, TransformerMixin
    
    # Получаем результаты из предыдущего выполнения
    selected_indices = [1, 2, 5, 7]
    print(f"Отобрано признаков: {len(selected_indices)}")
    print(f"Индексы отобранных признаков: {selected_indices}")
    
    # Создаем пайплайн с отобранными признаками
    class FeatureSelector(BaseEstimator, TransformerMixin):
        def __init__(self, indices):
            self.indices = indices
            
        def fit(self, X, y=None):
            return self
            
        def transform(self, X):
            return X[:, self.indices]
    
    # Подготовим данные для отбора
    preprocessor_simple = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), num_features),
            ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), cat_features),
        ],
        remainder='drop'
    )
    
    feature_selector_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor_simple),
        ('feature_selector', FeatureSelector(selected_indices)),
        ('model', CatBoostRegressor(random_state=42, verbose=0))
    ])
    
    # Логирование в MLflow
    with mlflow.start_run(run_name="feature_selection_mlxtend") as run:
        # Обучение модели
        print("Обучаем финальную модель с отобранными признаками...")
        feature_selector_pipeline.fit(X_train, y_train)
        
        # Предсказания и метрики
        predictions = feature_selector_pipeline.predict(X_test)
        metrics = {
            "mae": mean_absolute_error(y_test, predictions),
            "mape": mean_absolute_percentage_error(y_test, predictions),
            "mse": mean_squared_error(y_test, predictions),
            "rmse": np.sqrt(mean_squared_error(y_test, predictions))
        }
        
        # Получаем названия отобранных признаков
        all_features = num_features + cat_features
        selected_features = [all_features[i] for i in selected_indices]
        
        # Логирование параметров
        mlflow.log_params({
            'model_type': 'CatBoost_with_FeatureSelection',
            'selector': 'SequentialFeatureSelector',
            'direction': 'forward',
            'total_features': len(all_features),
            'selected_features': len(selected_indices),
            'selected_feature_names': str(selected_features)
        })
        
        # Логирование метрик
        mlflow.log_metrics(metrics)
        
        # Сохраняем информацию об отобранных признаках
        selection_info = {
            'total_features': len(all_features),
            'selected_features': len(selected_indices),
            'selected_indices': selected_indices,
            'selected_feature_names': selected_features,
            'all_features': all_features
        }
        
        import json
        with open("feature_selection_info.json", "w") as f:
            json.dump(selection_info, f, indent=2)
        mlflow.log_artifact("feature_selection_info.json")
        
        # Сохраняем названия признаков в отдельный файл
        with open("selected_feature_names.txt", "w") as f:
            for feature in selected_features:
                f.write(feature + "\n")
        mlflow.log_artifact("selected_feature_names.txt")
        
        print("="*50)
        print("Модель с отобранными признаками залогирована!")
        print(f"Отобрано {len(selected_features)} из {len(all_features)} признаков")
        print("Отобранные признаки:", selected_features)
        print("Метрики:")
        for metric, value in metrics.items():
            print(f" {metric.upper()}: {value:.4f}")
        print("="*50)

except Exception as e:
    print(f"Ошибка при логировании: {e}")

🏃 View run blushing-loon-957 at: http://localhost:5000/#/experiments/1/runs/f656252157bb417489efed5e01dd3682
🧪 View experiment at: http://localhost:5000/#/experiments/1
Отобрано признаков: 4
Индексы отобранных признаков: [1, 2, 5, 7]
Обучаем финальную модель с отобранными признаками...
Модель с отобранными признаками залогирована!
Отобрано 4 из 8 признаков
Отобранные признаки: ['Present_Price', 'Driven_kms', 'Fuel_Type', 'Transmission']
Метрики:
 MAE: 2.1102
 MAPE: 3.6306
 MSE: 9.1875
 RMSE: 3.0311
🏃 View run feature_selection_mlxtend at: http://localhost:5000/#/experiments/1/runs/6954fb37d93945d391ea51e453d6539e
🧪 View experiment at: http://localhost:5000/#/experiments/1


ПУНКТ 13: RFE ОТБОР ПРИЗНАКОВ

In [182]:


# Завершаем предыдущий run
mlflow.end_run()

from sklearn.feature_selection import RFE
from sklearn.base import BaseEstimator, TransformerMixin

class ColumnExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, indices):
        self.indices = indices
        
    def fit(self, X, y=None):
        return self
        
    def transform(self, X):
        return X[:, self.indices]

# Создаем препроцессор
preprocessor_rfe = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), cat_features),
    ],
    remainder='drop'
)

# Подготовим данные
print("Подготавливаем данные для RFE...")
X_train_preprocessed = preprocessor_rfe.fit_transform(X_train)

# Создаем RFE селектор
rfe_selector = RFE(
    estimator=CatBoostRegressor(random_state=42, verbose=0, iterations=100),
    n_features_to_select=4,  # выбираем 4 признака для сравнения с mlxtend
    step=0.2
)

# Обучаем RFE
print("Запускаем RFE отбор признаков...")
rfe_selector.fit(X_train_preprocessed, y_train)

# Получаем отобранные признаки
rfe_indices = rfe_selector.support_
selected_rfe_indices = [i for i, selected in enumerate(rfe_indices) if selected]
print(f"RFE отобрано признаков: {len(selected_rfe_indices)}")
print(f"RFE индексы отобранных признаков: {selected_rfe_indices}")

# Создаем пайплайн с RFE
rfe_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor_rfe),
    ('feature_selector', ColumnExtractor(selected_rfe_indices)),
    ('model', CatBoostRegressor(random_state=42, verbose=0))
])

# Логирование в MLflow
with mlflow.start_run(run_name="rfe_feature_selection") as run:
    # Обучение модели
    print("Обучаем модель с отобранными признаками...")
    rfe_pipeline.fit(X_train, y_train)
    
    # Предсказания и метрики
    predictions = rfe_pipeline.predict(X_test)
    metrics = {
        "mae": mean_absolute_error(y_test, predictions),
        "mape": mean_absolute_percentage_error(y_test, predictions),
        "mse": mean_squared_error(y_test, predictions),
        "rmse": np.sqrt(mean_squared_error(y_test, predictions))
    }
    
    # Получаем названия отобранных признаков
    all_features = num_features + cat_features
    selected_rfe_features = [all_features[i] for i in selected_rfe_indices]
    
    # Логирование параметров
    mlflow.log_params({
        'model_type': 'CatBoost_with_RFE',
        'selector': 'RFE',
        'total_features': len(all_features),
        'selected_features': len(selected_rfe_indices),
        'selected_feature_names': str(selected_rfe_features)
    })
    
    # Логирование метрик
    mlflow.log_metrics(metrics)
    
    # Сохраняем информацию об отобранных признаках
    rfe_info = {
        'total_features': len(all_features),
        'selected_features': len(selected_rfe_indices),
        'selected_indices': selected_rfe_indices,
        'selected_feature_names': selected_rfe_features
    }
    
    import json
    with open("rfe_selection_info.json", "w") as f:
        json.dump(rfe_info, f, indent=2)
    mlflow.log_artifact("rfe_selection_info.json")
    
    print("="*50)
    print("Модель с RFE отбором признаков залогирована!")
    print(f"Отобрано {len(selected_rfe_features)} из {len(all_features)} признаков")
    print("Отобранные признаки:", selected_rfe_features)
    print("Метрики:")
    for metric, value in metrics.items():
        print(f" {metric.upper()}: {value:.4f}")
    print("="*50)

Подготавливаем данные для RFE...
Запускаем RFE отбор признаков...
RFE отобрано признаков: 4
RFE индексы отобранных признаков: [0, 1, 6, 7]
Обучаем модель с отобранными признаками...
Модель с RFE отбором признаков залогирована!
Отобрано 4 из 8 признаков
Отобранные признаки: ['Year', 'Present_Price', 'Selling_type', 'Transmission']
Метрики:
 MAE: 2.3265
 MAPE: 4.0088
 MSE: 8.5653
 RMSE: 2.9267
🏃 View run rfe_feature_selection at: http://localhost:5000/#/experiments/1/runs/758977dc089c4197b70982ec84a53dad
🧪 View experiment at: http://localhost:5000/#/experiments/1


Пункт 14: Настройка гиперпараметров с Optuna:

In [183]:
print("=== ПУНКТ 14: OPTUNA НАСТРОЙКА ГИПЕРПАРАМЕТРОВ ===")

# Завершаем предыдущий run
mlflow.end_run()

import optuna

# Используем базовую модель для оптимизации
def objective(trial):
    # Предлагаем гиперпараметры
    depth = trial.suggest_int('depth', 4, 8)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.2)
    iterations = trial.suggest_int('iterations', 500, 1500)
    l2_leaf_reg = trial.suggest_float('l2_leaf_reg', 1, 5)
    
    # Создаем модель
    model = CatBoostRegressor(
        depth=depth,
        learning_rate=learning_rate,
        iterations=iterations,
        l2_leaf_reg=l2_leaf_reg,
        random_state=42,
        verbose=0
    )
    
    # Создаем пайплайн
    optuna_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    
    # Обучаем модель
    optuna_pipeline.fit(X_train, y_train)
    
    # Предсказания и метрика
    predictions = optuna_pipeline.predict(X_test)
    mae = mean_absolute_error(y_test, predictions)
    
    return mae

# Создаем исследование
study = optuna.create_study(direction='minimize')
print("Запускаем Optuna оптимизацию (10 trials)...")
study.optimize(objective, n_trials=10)

# Выводим результаты
print('Количество завершенных trials:', len(study.trials))
print('Лучший trial:')
best_params = study.best_trial.params
print(best_params)

# Обучаем финальную модель с лучшими параметрами
best_model = CatBoostRegressor(**best_params, random_state=42, verbose=0)
best_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', best_model)
])

best_pipeline.fit(X_train, y_train)

# Предсказания и метрики
predictions = best_pipeline.predict(X_test)
metrics = {
    "mae": mean_absolute_error(y_test, predictions),
    "mape": mean_absolute_percentage_error(y_test, predictions),
    "mse": mean_squared_error(y_test, predictions),
    "rmse": np.sqrt(mean_squared_error(y_test, predictions))
}

# Логирование в MLflow
with mlflow.start_run(run_name="optuna_tuned_model") as run:
    mlflow.log_params(best_params)
    mlflow.log_params({
        'model_type': 'CatBoost_Optuna_Tuned',
        'optuna_trials': 10
    })
    
    mlflow.log_metrics(metrics)
    
    # Логируем модель
    mlflow.sklearn.log_model(
        best_pipeline,
        artifact_path="optuna_tuned_model",
        registered_model_name="estate_model_optuna"
    )
    
    print("="*50)
    print("Optuna-оптимизированная модель залогирована и зарегистрирована!")
    print("Лучшие параметры:", best_params)
    print("Метрики:")
    for metric, value in metrics.items():
        print(f" {metric.upper()}: {value:.4f}")
    print("="*50)

=== ПУНКТ 14: OPTUNA НАСТРОЙКА ГИПЕРПАРАМЕТРОВ ===


[I 2025-11-20 03:40:14,109] A new study created in memory with name: no-name-830276ec-7407-4bf2-8808-376255f5b851


Запускаем Optuna оптимизацию (10 trials)...


[I 2025-11-20 03:40:14,654] Trial 0 finished with value: 1.6068234563232358 and parameters: {'depth': 6, 'learning_rate': 0.10234003308492803, 'iterations': 726, 'l2_leaf_reg': 3.616384689789866}. Best is trial 0 with value: 1.6068234563232358.
[I 2025-11-20 03:40:15,089] Trial 1 finished with value: 1.5465995741547347 and parameters: {'depth': 7, 'learning_rate': 0.022793599416938773, 'iterations': 650, 'l2_leaf_reg': 4.273328119606635}. Best is trial 1 with value: 1.5465995741547347.
[I 2025-11-20 03:40:15,905] Trial 2 finished with value: 1.3665659628676725 and parameters: {'depth': 6, 'learning_rate': 0.168585963571356, 'iterations': 1240, 'l2_leaf_reg': 4.634330990933859}. Best is trial 2 with value: 1.3665659628676725.
[I 2025-11-20 03:40:16,405] Trial 3 finished with value: 1.1795221306183268 and parameters: {'depth': 5, 'learning_rate': 0.13057928186491596, 'iterations': 1007, 'l2_leaf_reg': 4.6981759219124335}. Best is trial 3 with value: 1.1795221306183268.
[I 2025-11-20 03:4

Количество завершенных trials: 10
Лучший trial:
{'depth': 5, 'learning_rate': 0.13057928186491596, 'iterations': 1007, 'l2_leaf_reg': 4.6981759219124335}


2025/11/20 03:40:21 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/20 03:40:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'estate_model_optuna'.
2025/11/20 03:40:30 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: estate_model_optuna, version 1
Created version '1' of model 'estate_model_optuna'.


Optuna-оптимизированная модель залогирована и зарегистрирована!
Лучшие параметры: {'depth': 5, 'learning_rate': 0.13057928186491596, 'iterations': 1007, 'l2_leaf_reg': 4.6981759219124335}
Метрики:
 MAE: 1.1795
 MAPE: 2.4339
 MSE: 2.3532
 RMSE: 1.5340
🏃 View run optuna_tuned_model at: http://localhost:5000/#/experiments/1/runs/7128fb3e90714e638a3ebeca4d0db07a
🧪 View experiment at: http://localhost:5000/#/experiments/1


Пункт 15: CatBoost с нативной обработкой категориальных признаков:

In [ ]:
print("=== ПУНКТ 15: CATBOOST С НАТИВНОЙ ОБРАБОТКОЙ КАТЕГОРИАЛЬНЫХ ПРИЗНАКОВ ===")

# Завершаем предыдущий run
mlflow.end_run()

# Используем CatBoost с нативной поддержкой категориальных признаков
# Для этого нам нужно преобразовать категориальные признаки в строковый тип
X_train_catboost = X_train.copy()
X_test_catboost = X_test.copy()

# Преобразуем категориальные признаки в строки для CatBoost
for col in cat_features:
    X_train_catboost[col] = X_train_catboost[col].astype(str)
    X_test_catboost[col] = X_test_catboost[col].astype(str)

catboost_pipeline = Pipeline(steps=[
    ('model', CatBoostRegressor(
        cat_features=cat_features,  # CatBoost сам обработает категориальные признаки
        random_state=42,
        verbose=0
    ))
])

# Обучаем модель
print("Обучаем CatBoost с нативной обработкой категориальных признаков...")
catboost_pipeline.fit(X_train_catboost, y_train)

# Предсказания и метрики
predictions = catboost_pipeline.predict(X_test_catboost)
metrics = {
    "mae": mean_absolute_error(y_test, predictions),
    "mape": mean_absolute_percentage_error(y_test, predictions),
    "mse": mean_squared_error(y_test, predictions),
    "rmse": np.sqrt(mean_squared_error(y_test, predictions))
}

# Логирование в MLflow с CatBoost flavour
with mlflow.start_run(run_name="catboost_native") as run:
    mlflow.log_params({
        'model_type': 'CatBoost_Native',
        'used_cat_features': str(cat_features),
        'native_categorical_handling': True
    })
    
    mlflow.log_metrics(metrics)
    
    # Логируем модель с CatBoost flavour
    mlflow.catboost.log_model(
        cb_model=catboost_pipeline.named_steps['model'],
        artifact_path="catboost_native_model",
        registered_model_name="car_price_catboost_native"
    )
    

    
    print("="*50)
    print("CatBoost модель с нативной обработкой категориальных признаков залогирована!")
    print("Метрики:")
    for metric, value in metrics.items():
        print(f" {metric.upper()}: {value:.4f}")
    print("="*50)

=== ПУНКТ 15: CATBOOST С НАТИВНОЙ ОБРАБОТКОЙ КАТЕГОРИАЛЬНЫХ ПРИЗНАКОВ ===
Обучаем CatBoost с нативной обработкой категориальных признаков...


2025/11/20 03:42:33 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/20 03:42:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'car_price_catboost_native'.
2025/11/20 03:42:44 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: car_price_catboost_native, version 1
Created version '1' of model 'car_price_catboost_native'.


CatBoost модель с нативной обработкой категориальных признаков залогирована!
Метрики:
 MAE: 2.2819
 MAPE: 3.9572
 MSE: 7.8919
 RMSE: 2.8092
🏃 View run catboost_native at: http://localhost:5000/#/experiments/1/runs/018f58e373584ede8509a5ffe308e5b0
🧪 View experiment at: http://localhost:5000/#/experiments/1


In [195]:
from mlflow.tracking import MlflowClient

# Простой и надежный способ
def simple_promote_to_production():
    client = MlflowClient()
    
    # Сначала посмотрим, какие версии есть у модели
    versions = client.search_model_versions("name='baseline_final'")
    
    print(" Доступные версии модели 'baseline_final':")
    for v in versions:
        print(f"   Версия {v.version}: {v.current_stage} (Run: {v.run_id})")
    
    if versions:
        # Берем последнюю версию
        latest_version = max([int(v.version) for v in versions])
        
        print(f"\n Перевод версии {latest_version} в Production...")
        
        client.transition_model_version_stage(
            name="baseline_final",
            version=latest_version,
            stage="Production"
        )
        
        print(f" baseline_final v{latest_version} успешно в Production!")
    else:
        print("Нет версий модели 'baseline_final'")

# Выполняем
simple_promote_to_production()

 Доступные версии модели 'baseline_final':
   Версия 2: Production (Run: 2d965e2930644df18cbd649bb52cdaed)
   Версия 1: None (Run: 2d965e2930644df18cbd649bb52cdaed)

 Перевод версии 2 в Production...
 baseline_final v2 успешно в Production!


/tmp/ipykernel_4373/2384721826.py:20: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


In [ ]:
print("=== ПУНКТ 13: RFE ОТБОР ПРИЗНАКОВ ===")

from sklearn.feature_selection import RFE
from sklearn.base import BaseEstimator, TransformerMixin

class ColumnExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, indices):
        self.indices = indices
        
    def fit(self, X, y=None):
        return self
        
    def transform(self, X):
        return X[:, self.indices]

# Создаем препроцессор
preprocessor_rfe = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), cat_features),
    ],
    remainder='drop'
)

# Подготовим данные
X_train_preprocessed = preprocessor_rfe.fit_transform(X_train)

# Создаем RFE селектор с простой моделью для скорости
rfe_selector = RFE(
    estimator=CatBoostRegressor(random_state=42, verbose=0, iterations=50),
    n_features_to_select=min(8, len(num_features + cat_features)),  # выбираем 8 признаков или меньше
    step=0.2
)

# Обучаем RFE
print("Запускаем RFE отбор признаков...")
rfe_selector.fit(X_train_preprocessed, y_train)

# Получаем отобранные признаки
rfe_indices = rfe_selector.support_
selected_rfe_indices = [i for i, selected in enumerate(rfe_indices) if selected]
print(f"RFE отобрано признаков: {len(selected_rfe_indices)}")
print(f"RFE индексы отобранных признаков: {selected_rfe_indices}")

# Создаем пайплайн с RFE
rfe_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor_rfe),
    ('feature_selector', ColumnExtractor(selected_rfe_indices)),
    ('model', CatBoostRegressor(random_state=42, verbose=0))
])

# Логирование в MLflow
with mlflow.start_run(run_name="rfe_feature_selection") as run:
    # Обучение модели
    rfe_pipeline.fit(X_train, y_train)
    
    # Предсказания и метрики
    predictions = rfe_pipeline.predict(X_test)
    metrics = {
        "mae": mean_absolute_error(y_test, predictions),
        "mape": mean_absolute_percentage_error(y_test, predictions),
        "mse": mean_squared_error(y_test, predictions),
        "rmse": np.sqrt(mean_squared_error(y_test, predictions))
    }
    
    # Получаем названия отобранных признаков
    all_features = num_features + cat_features
    selected_rfe_features = [all_features[i] for i in selected_rfe_indices]
    
    # Логирование параметров
    mlflow.log_params({
        'model_type': 'CatBoost_with_RFE',
        'selector': 'RFE',
        'total_features': len(all_features),
        'selected_features': len(selected_rfe_indices),
        'selected_feature_names': str(selected_rfe_features)
    })
    
    # Логирование метрик
    mlflow.log_metrics(metrics)
    
    # Сохраняем информацию об отобранных признаках
    rfe_info = {
        'total_features': len(all_features),
        'selected_features': len(selected_rfe_indices),
        'selected_indices': selected_rfe_indices,
        'selected_feature_names': selected_rfe_features
    }
    
    import json
    with open("rfe_selection_info.json", "w") as f:
        json.dump(rfe_info, f, indent=2)
    mlflow.log_artifact("rfe_selection_info.json")
    
    print("Модель с RFE отбором признаков залогирована!")
    print("Отобранные признаки:", selected_rfe_features)
    print("Метрики:")
    for metric, value in metrics.items():
        print(f" {metric.upper()}: {value:.4f}")

=== ПУНКТ 12: ОТБОР ПРИЗНАКОВ MLXTEND ===
MLxtend не установлен: No module named 'mlxtend'
Установите: pip install mlxtend


VSE CHTO NIZE NE NUZNO

In [118]:
print("=== ПУНКТ 12: ОТБОР ПРИЗНАКОВ MLXTEND ===")

try:
    from mlxtend.feature_selection import SequentialFeatureSelector
    from sklearn.base import BaseEstimator, TransformerMixin
    
    # Используем базовые признаки
    print("Используем базовые признаки...")
    
    # Вычисляем N - от 20% до 70% признаков
    total_features = len(num_features + cat_features)
    n_features = max(1, int(total_features * 0.5))  # берем 50%
    print(f"Всего признаков: {total_features}, отбираем: {n_features}")
    
    # Создаем простую модель для отбора признаков
    simple_model = CatBoostRegressor(random_state=42, verbose=0, iterations=100)
    
    # Подготовим данные для отбора
    preprocessor_simple = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), num_features),
            ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), cat_features),
        ],
        remainder='drop'
    )
    
    X_train_preprocessed = preprocessor_simple.fit_transform(X_train)
    
    # Создаем селектор признаков с меньшим количеством cv для скорости
    sfs = SequentialFeatureSelector(
        simple_model,
        k_features=n_features,
        forward=True,
        floating=False,
        verbose=1,
        scoring='neg_mean_absolute_error',
        cv=2,  # уменьшаем для скорости
        n_jobs=1  # убираем параллелизацию для стабильности
    )
    
    # Обучаем селектор
    print("Запускаем отбор признаков...")
    sfs.fit(X_train_preprocessed, y_train)
    
    # Получаем отобранные признаки
    selected_indices = list(sfs.k_feature_idx_)
    print(f"Отобрано признаков: {len(selected_indices)}")
    print(f"Индексы отобранных признаков: {selected_indices}")
    
    # Создаем пайплайн с отобранными признаками
    class FeatureSelector(BaseEstimator, TransformerMixin):
        def __init__(self, indices):
            self.indices = indices
            
        def fit(self, X, y=None):
            return self
            
        def transform(self, X):
            return X[:, self.indices]
    
    feature_selector_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor_simple),
        ('feature_selector', FeatureSelector(selected_indices)),
        ('model', CatBoostRegressor(random_state=42, verbose=0))
    ])
    
    # Логирование в MLflow
    with mlflow.start_run(run_name="feature_selection_mlxtend") as run:
        # Обучение модели
        feature_selector_pipeline.fit(X_train, y_train)
        
        # Предсказания и метрики
        predictions = feature_selector_pipeline.predict(X_test)
        metrics = {
            "mae": mean_absolute_error(y_test, predictions),
            "mape": mean_absolute_percentage_error(y_test, predictions),
            "mse": mean_squared_error(y_test, predictions),
            "rmse": np.sqrt(mean_squared_error(y_test, predictions))
        }
        
        # Получаем названия отобранных признаков
        all_features = num_features + cat_features
        selected_features = [all_features[i] for i in selected_indices]
        
        # Логирование параметров
        mlflow.log_params({
            'model_type': 'CatBoost_with_FeatureSelection',
            'selector': 'SequentialFeatureSelector',
            'direction': 'forward',
            'total_features': total_features,
            'selected_features': len(selected_indices),
            'selected_feature_names': str(selected_features)
        })
        
        # Логирование метрик
        mlflow.log_metrics(metrics)
        
        # Сохраняем информацию об отобранных признаках
        selection_info = {
            'total_features': total_features,
            'selected_features': len(selected_indices),
            'selected_indices': selected_indices,
            'selected_feature_names': selected_features
        }
        
        import json
        with open("feature_selection_info.json", "w") as f:
            json.dump(selection_info, f, indent=2)
        mlflow.log_artifact("feature_selection_info.json")
        
        print("Модель с отобранными признаками залогирована!")
        print("Отобранные признаки:", selected_features)
        print("Метрики:")
        for metric, value in metrics.items():
            print(f" {metric.upper()}: {value:.4f}")

except ImportError as e:
    print(f"MLxtend не установлен: {e}")
    print("Установите: pip install mlxtend")
except Exception as e:
    print(f"Ошибка при работе с MLxtend: {e}")
    print("Пропускаем пункт 12 и переходим к следующему.")

=== ПУНКТ 12: ОТБОР ПРИЗНАКОВ MLXTEND ===
MLxtend не установлен: No module named 'mlxtend'
Установите: pip install mlxtend


In [ ]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor), 
    ('model', regressor)
])


In [ ]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor), 
                           ('model', regressor)])

pipeline.fit(X_train, y_train)

,steps,"[('preprocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [ ]:
# Создаем pipeline для РЕГРЕССИИ
pipeline_reg = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(
        n_estimators=100,  # Можно уменьшить до 50 если обучение долгое
        max_depth=10,      # Можно уменьшить до 5-7 если обучение долгое
        random_state=42,
        n_jobs=-1# Создаем pipeline для РЕГРЕССИИ
pipeline_reg = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(
        n_estimators=100,  # Можно уменьшить до 50 если обучение долгое
        max_depth=10,      # Можно уменьшить до 5-7 если обучение долгое
        random_state=42,
        n_jobs=-1
    ))
])

print("Pipeline для регрессии создан")
print(pipeline_reg)
    ))
])

print("Pipeline для регрессии создан")
print(pipeline_reg)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (796817068.py, line 8)